# Combined Datasets

In [ ]:
# Importing dependencies
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score

# Data

Data explanation placeholder

(Talk about original two notebooks?)

In [ ]:
# Reading in data
df_movies = pd.read_csv("./Resources/movies_data.csv")

### Movie Data

# Economics Data

While classifying economic states is a complex and nuanced issue, it is not unreasonable to draw more broad-strokes generalizations about a given timeframe based on more limited factors. To serve the purposes of our modelling, the following three factors were chosen to highlight the economic status at a given movie's release date;

* Consumer Confidence Indicator (CCI)
* Consumer Price Index (CPI)
* Unemployment Rate

These features are further discussed below, and stand as adequate datapoints to answer three respective questions;

* How likely are people to be spending money?
* How much do things cost when they do spend money?
* How many people have jobs to earn money to spend?

As detailed below, this information came as monthly measures over several decades. To create our `Economic Climate` indicator - a classification as to whether or not the economics of a given time were on the better side for consumers - we will need to calculate a rolling 12-month percent change in the mean of those monthly values in order to show if a given feature was on a positive or negative trend for the provided period.

---

The following datasets are courtesy of __[Kaggle](https://www.kaggle.com/)__.

### __['...CCI_OECD.csv'](https://www.kaggle.com/datasets/iqbalsyahakbar/cci-oecd)__

*renamed from `DP_LIVE_16112023095843236.csv`*

Per the Organisation for Economic Co-operation and Development (OECD);

* The CCI is an indication of developments for future households' consumption and saving based on expected financial situation, sentiment regarding the general economic situation, employment status, and capacity for savings
* An indicator above `100` indicates an optimistic outlook and a greater likliehood to spend money over cautious saving
* An indicator below `100` indicates a pessimistic outlook and both a higher likeliehood to save money and a lower tendency to consume

### __['...US_inflation_rates.csv'](https://www.kaggle.com/datasets/pavankrishnanarne/us-inflation-dataset-1947-present)__

Per the dataset description;

* The CPI is a critical economic indicator for measuring the purchasing power of money over time, measuring the average change over time in the prices paid by urban consumers for goods and services
* The CPI is the value at the end of the respective month

---

The following datasets are courtesy of the __[Economic Policy Institute’s (EPI) State of Working America Data Library](https://www.epi.org/data/)__.

### __['...Unemployment.csv'](https://www.epi.org/data/#?subject=unemp)__

Per EPI description;

* Unemployment is the share of the labor force wihout a job
* Monthly percentages calculated as a rolling 12-month average (mean)

In [ ]:
# Reading in data
df_unemp = pd.read_csv("./Resources/EPI Data Library - Unemployment.csv")
df_cci = pd.read_csv("./Resources/CCI_OECD.csv")
df_inflation = pd.read_csv("./Resources/US_inflation_rates.csv")

## Defining functions

Since each dataset will need similar preprocessing, the following functions will be used to help streamline the flow and code.

### Universal functions

Applicable to all datasets

#### EDA routine

Labelling and displaying pertinant information about a given dataset for the purposes of expedited EDA

#### Copying datasets

Creating a working copy of a given dataset to preserve the original DF with unneeded features dropped

#### Renaming needed features

Renaming selected features for a given dataset

#### Rolling mean and mean percent change

Calculating the rolling 12-month mean and the rolling 12-month percent change for a given feature

In [ ]:
# Defining a function to display the `.describe()`, `.shape`, and `.dtypes`
# for a given DF
def eda_routine(df):
    print('Describe:')
    display(df.describe())
    print(f'Shape: {df.shape}\n')
    print(f'Data types:')
    display(df.dtypes)

# Defining a function to copy a dataset with only the needed features
def copy_df(df, features_to_keep):
    df_copy = df[features_to_keep].copy()
    return df_copy

# Defining a function to rename needed features
def rename_features(df, feature1, feature1new, feature2, feature2new):
    df.rename(columns={
        feature1: feature1new,
        feature2: feature2new
    }, inplace=True)
    return df

# Defining a function to calculate the rolling 12-month means and percent changes
# for a given feature
def rolling_calcs(df, feature, feature_mean, feature_pct_chng):
    df[feature_mean] = df[feature].rolling(window=12).mean()
    df[feature_pct_chng] = df[feature_mean].pct_change(periods=12) * 100
    return df

### Situational functions

Applicable to select datasets

#### Datetime indexing

Converting the feature containing the raw datetime information into a suitable datetime index

*Cannot be used on `Unemployment` dataset*

#### Removing '%'

Removing the `'%'` from a given feature and converting the remaining `object` dtype to `float`

*Specigically for `Unemployment` dataset*

In [ ]:
# Defining a function to set a `Date` feature as a datetime index
def datetime_index(df, datetime_feature):
    df[datetime_feature] = pd.to_datetime(df[datetime_feature])
    df.set_index(datetime_feature, inplace=True)
    df.sort_index(inplace=True)
    return df

# Defining a function to remove '%' and convert data `float`
def convert_percentage(feature):
    return float(feature.strip('%'))

# Defining a function to apply `convert_percentage`
def apply_percentage(df, feature):
    df[feature] = df[feature].apply(convert_percentage)
    return df

## CCI

#### Preprocessing of the `CCI_OECD.csv` dataset

This dataset came with internaitonal records and unneeded features, so only records for US CCI will be retained. Once those records have been selected, the resulting DF will need to be prepared for concatenation with the remainined economic datasets. To do this, the `TIME` feature will be converted to datetime and set as the index.

In [ ]:
# Viewing `df_cci`
df_cci.head()

#### Beginning of limited EDA

In [ ]:
# Applying `eda_routine` to `df_cci`
eda_routine(df_cci)

In [ ]:
# Continuing EDA
df_cci['LOCATION'].unique()

#### Selecting only domestic data

In [ ]:
# Copying domestic data from `df_cci` to `df_cci_us` and removing unneeded features
df_cci_us = df_cci.loc[df_cci['LOCATION'] == 'USA'].copy()

#### Applying defined functions

In [ ]:
# Copying `df_cci_us` and dropping unneeded features
df_cci_form = copy_df(df_cci_us, ['TIME', 'Value'])

# Renamining retained features
df_cci_form = rename_features(
    df_cci_form, 'TIME', 'Date', 'Value', 'CCI Value'
)

# Converting `Date` to a datetime index
df_cci_form = datetime_index(df_cci_form, 'Date')

# Calculating rolling 12-month means and percent change in means
df_cci_form = rolling_calcs(
    df_cci_form, 'CCI Value', 'CCI Rolling Mean', 'CCI Rolling Percent Change'
)

# Confirming `df_cci_form` ready to concatenate
display(df_cci_form.head())
display(df_cci_form.tail())

## Inflation

#### Preprocessing of the `US_inflation_rates.csv` dataset

Seeing as the dataset came with only the needed features, little will be needed to prepare the DF for concatenation with the other economic datasets. `date` will be converted to datetime and set as the index.

In [ ]:
# Viewing `df_inflation`
df_inflation.head()

#### Beginning of limited EDA

In [ ]:
# Applying `eda_routine` to `df_inflation`
eda_routine(df_inflation)

#### Applying defined functions

In [ ]:
# Copying `df_inflation` and dropping unneeded features
df_inflation_form = copy_df(df_inflation, ['date', 'value'])

# Renamining retained features
df_inflation_form = rename_features(
    df_inflation_form, 'date', 'Date', 'value', 'CPI Value'
)

# Converting `Date` to a datetime index
df_inflation_form = datetime_index(df_inflation_form, 'Date')

# Calculating rolling 12-month means and percent change in means
df_inflation_form = rolling_calcs(
    df_inflation_form,
    'CPI Value',
    'CPI Rolling Mean',
    'CPI Rolling Percent Change'
)

# Confirming `df_inflation_form` ready to concatenate
display(df_inflation_form.head())
display(df_inflation_form.tail())

## Unemployment

#### Preprocessing of the `Unemployment.csv` dataset

This dataset came with unneeded features that will need to be dropped, as well as the needed features will need to be converted to `float`. Additionally, the `Date` feature will need to be converted to datetime and set to the index in preparation for concatenation with the other economic datasets.

In [ ]:
# Viewing `df_unemp`
df_unemp.head()

#### Beginning of limited EDA

In [ ]:
# Applying `eda_routine` to `df_unemp`
eda_routine(df_unemp)

#### Applying defined functions (first pass)

*Given the nature of the* `Date` *feature in this dataset, the datetime indexing will need to be handled outside of the defined functions*

In [ ]:
# Copying `df_unemp` and dropping unneeded features
df_unemp_form = copy_df(df_unemp, ['Date', 'All'])

#### Renaming needed feature

*This dataset only needed one feature,* `All`*, to be renmaned, this the* `rename_features` *defined function is not applicable*

In [ ]:
# Renaming the reatined feature
df_unemp_form.rename(columns={'All': 'Unemployment Rate (%)'}, inplace=True)

#### Datetime indexing

In [ ]:
# The `Date` feature will need to be engineered into a workable datetime feature

# Creating a dictionary of Months
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Mapping integer month values to `Date Month`
df_unemp_form['Date Month'] = df_unemp_form['Date'].str.slice(0,3).map(month_map)

# Slicing `Date Year`
df_unemp_form['Date Year'] = df_unemp_form['Date'].str.slice(4,8)

# Converting `Date` to datetime using `Date Month` and `Date Year`
df_unemp_form['Date'] = pd.to_datetime({
    'year': df_unemp_form['Date Year'],
    'month': df_unemp_form['Date Month'],
    'day': 1
})

# Dropping engineered features `Date Month` and `Date Year`
df_unemp_form.drop(columns=['Date Month', 'Date Year'], inplace=True)

# Setting `Date` as index
df_unemp_form.set_index('Date', inplace=True)

# Ensuring index is sorted with ascending dates
df_unemp_form.sort_index(inplace=True)

#### Applying defined functions (second pass)

In [ ]:
# Applying `apply_percentage` to `Unemployment Rate (%)`
df_unemp_form = apply_percentage(df_unemp_form, 'Unemployment Rate (%)')

# Calculating rolling 12-month means and percent change in means
df_unemp_form = rolling_calcs(
    df_unemp_form,
    'Unemployment Rate (%)',
    'Unemployment Rate (%) Rolling Mean',
    'Unemployment Rate Rolling Percent Change',
)

# Confirming `df_unemp_form` ready to concatenate
display(df_unemp_form.head())
display(df_unemp_form.tail())

## Combined Economics

#### Preprocessing of the `df_economics` DF

With all datasets set to a monthly datetime index, the relevent features of all can be combined into one DF, and any NaN records can be dropped.

In [ ]:
# Reconfirming total records and features for datasets
df_cci_form.shape, df_inflation_form.shape, df_unemp_form.shape

In [ ]:
# Concatenating the economic datasets into `df_economics`
df_economics = pd.concat(
    [
        df_cci_form,
        df_inflation_form,
        df_unemp_form
    ], axis=1, join='outer'
)

#### Handling `NaN` rows

In [ ]:
# Confirming total records and features
df_economics.shape

In [ ]:
# Checking total `NaN` records
df_economics.isna().count()

In [ ]:
# Dropping `NaN` records
df_economics.dropna(inplace=True)

# Confirming remaining records
df_economics.shape[0]

In [ ]:
# Confirming final economic DF
display(df_economics.head())
display(df_economics.tail())

## Engineering

#### Engineering the economic target value

As stated, the goal is to create an indicator for `Economic Climate` based on broad-strokes observations of our datasets. Having calculated the rolling 12-month percent change for each feature - based off the rolling 12-month mean - we can look for a positive or negative change in values and flag the movement accordingly. From there, we can make the following simple statements;

* For **CCI**, a positive change is "good", as it indicates an increase in the likelihood of consumers to spend money
* For **CPI**, a negative change is "good", as it indicates a decrease in the costs for goods and services
* For **Unemployment Rate**, a negative change is "good", as it indicates an incrase in the population active in the workforce

Therefore, we can interpret movement contrary to those changes as "bad". With this simplified view of the features, we can draw a classification as follows;

* If **at least two (2) features** have a "good" value, we can set `Economic Climate` to `Comfortable to Good`
* If **at least two (2) features** have a "bad" value, we can set `Economic Climate` to `Lean to Bad`

In this way, we can gague whether the ecnomic state at a given rlease date supports or disproves our hypothesis.

#### Continued EDA

In [ ]:
# Continuing EDA
df_economics.describe()

#### Classifying `Economic Climate`

In [ ]:
# Creating a list of features
features_to_flag = [
    'CCI Rolling Percent Change',
    'CPI Rolling Percent Change',
    'Unemployment Rate Rolling Percent Change'
]

# Looping through `features_to_flag` to assign `positive` and `negative` indicators
for col in df_economics[features_to_flag].columns:
    new_col = str(col) + ' Flag'
    df_economics.loc[df_economics[col] > 0, new_col] = 'positive'
    df_economics.loc[df_economics[col] <= 0, new_col] = 'negative'

# Creating a list flagged features
flag_cols = [
    'CCI Rolling Percent Change Flag',
    'CPI Rolling Percent Change Flag',
    'Unemployment Rate Rolling Percent Change Flag'
]

# Confirming indicators applied
df_economics[flag_cols].head()

In [ ]:
# Creating a list of conditions and classifications
conditions = [
    ((df_economics[flag_cols[0]] == 'positive') &   # CCI = 'positive'/good
    (df_economics[flag_cols[1]] == 'positive') &    # CPI = 'positive'/bad
    (df_economics[flag_cols[2]] == 'positive'),     # Unemplyment = 'positive'/bad
    'Lean to Bad'),
    ((df_economics[flag_cols[0]] == 'positive') &   # CCI = 'positive'/good
    (df_economics[flag_cols[1]] == 'positive') &    # CPI = 'positive'/bad
    (df_economics[flag_cols[2]] == 'negative'),     # Unemplyment = 'negative'/good
    'Comfortable to Good'),
    ((df_economics[flag_cols[0]] == 'positive') &   # CCI = 'positive'/good
    (df_economics[flag_cols[1]] == 'negative') &    # CPI = 'negative'/good
    (df_economics[flag_cols[2]] == 'positive'),     # Unemplyment = 'positive'/bad
    'Comfortable to Good'),
    ((df_economics[flag_cols[0]] == 'negative') &   # CCI = 'negative'/bad
    (df_economics[flag_cols[1]] == 'positive') &    # CPI = 'positive'/bad
    (df_economics[flag_cols[2]] == 'positive'),     # Unemplyment = 'positive'/bad
    'Lean to Bad'),
    ((df_economics[flag_cols[0]] == 'negative') &   # CCI = 'negative'/bad
    (df_economics[flag_cols[1]] == 'negative') &    # CPI = 'negative'/good
    (df_economics[flag_cols[2]] == 'positive'),     # Unemplyment = 'positive'/bad
    'Lean to Bad'),
    ((df_economics[flag_cols[0]] == 'negative') &   # CCI = 'negative'/bad
    (df_economics[flag_cols[1]] == 'positive') &    # CPI = 'positive'/bad
    (df_economics[flag_cols[2]] == 'negative'),     # Unemplyment = 'negative'/good
    'Lean to Bad'),
    ((df_economics[flag_cols[0]] == 'positive') &   # CCI = 'positive'/good
    (df_economics[flag_cols[1]] == 'negative') &    # CPI = 'negative'/good
    (df_economics[flag_cols[2]] == 'negative'),     # Unemplyment = 'negative'/good
    'Comfortable to Good'),
    ((df_economics[flag_cols[0]] == 'negative') &   # CCI = 'negative'/bad
    (df_economics[flag_cols[1]] == 'negative') &    # CPI = 'negative'/good
    (df_economics[flag_cols[2]] == 'negative'),     # Unemplyment = 'negative'/good
    'Comfortable to Good')
]

# Declaring `Economic Climate` with a `PLACEHOLDER` value
df_economics['Economic Climate'] = 'PLACEHOLDER'

# Applying conditions and classifications to `Economic Climate`
for condition, classification in conditions:
    df_economics.loc[condition, 'Economic Climate'] = classification

# Confirming classifications applied
df_economics['Economic Climate'].value_counts()

## Visualizations

In [ ]:
# Creating a list of features
features_to_plot = [
    'CCI Value',
    'CPI Value',
    'Unemployment Rate (%)'
]

# Creating a value of `0` to show positive and negative values
zero_line = pd.DataFrame({
    'Date': df_economics.index,
    'val': [x for x in 0*df_economics[features_to_flag[2]]]
})
zero_line.set_index('Date', inplace=True)

# Creating a value of `100` to show break point for CCI
hundred_line = pd.DataFrame({
    'Date': df_economics.index,
    'val': [x for x in (0*df_economics[features_to_flag[0]])+100]
})
hundred_line.set_index('Date', inplace=True)

In [ ]:
# Visualizing trends for `CCI Value`
plt.plot(hundred_line, color='black', linestyle='--')
plt.plot(df_economics[features_to_plot[0]], label='CCI', color='blue')
plt.title('Values above 100 (visualized)\n indicate consumers more likely to spend vs save')
plt.legend()
plt.show()

In [ ]:
# Visualizing trends for `CCI Rolling Percent Change`
plt.plot(zero_line, color='black', linestyle='--')
plt.plot(df_economics[features_to_flag[0]], label='% Changes in CCI', color='blue')
plt.legend()
plt.show()

In [ ]:
# Visualizing trends for `CPI Value`
plt.plot(df_economics[features_to_plot[1]], label='CPI', color='red')
plt.legend()
plt.show()

In [ ]:
# Visualizing trends for `CPI Rolling Percent Change`
plt.plot(zero_line, color='black', linestyle='--')
plt.plot(df_economics[features_to_flag[1]], label='% Changes in CPI', color='red')
plt.legend()
plt.show()

In [ ]:
# Visualizing trends for `Unemployment Rate (%)`
plt.plot(df_economics[features_to_plot[2]], label='Unemployment Rate (%)', color='red')
plt.legend()
plt.show()

In [ ]:
# Visualizing trends for `Unemployment Rate Rolling Percent Change`
plt.plot(zero_line, color='black', linestyle='--')
plt.plot(df_economics[features_to_flag[2]], label='% Changes in Unemployment Rate', color='red')
plt.legend()
plt.show()

In [ ]:
# Vizualizing total years classified in `Economic Climate`
plt.barh(
    y=df_economics['Economic Climate'].value_counts().index,
    width=df_economics['Economic Climate'].value_counts()/12,
    color=['darkblue', 'darkgreen'],
    label=['26.42', '16.25']
)
plt.title(
    'Years from 1981 to 2023 Classified as',
    loc='left',
    pad=15
)
plt.legend()
plt.show()

## Indexing

In [ ]:
# Resetting the index to recreate `Date` for later concatenation
df_economics.reset_index(inplace=True)

# Combining Data

In [ ]:
# Creating a 'Date' for a datetime index
df_movies['Date'] = pd.to_datetime({
    'year': df_movies['released_year'],
    'month': df_movies['released_month'],
    'day': df_movies['released_day']
})

# Setting `Date` as index
df_movies.set_index('Date', inplace=True)

# Ensuring index is sorted with ascending dates
df_movies.sort_index(inplace=True)

In [ ]:
# Creating a `Year` and `Month` for concatenation
df_economics['Year'] = df_economics['Date'].dt.strftime('%Y').astype(int)
df_economics['Month'] = df_economics['Date'].dt.strftime('%m').astype(int)

# Renaming to `Year` and `Month` for concatenation
df_movies.rename(columns={
'released_year': 'Year',
'released_month': 'Month'
}, inplace=True)

In [ ]:
# Confirming total records before concatenation
print(f'Total ecomonic records: {df_economics.shape[0]}')
print(f'Total movie records: {df_movies.shape[0]}')

In [ ]:
# Combining datasets through concatenation
df_combined = pd.merge(df_economics, df_movies, how='left', on=['Year', 'Month'])

# Confirming total records after concatenation
print(f'Total records: {df_combined.shape[0]}')

# EDA

In [ ]:
# Creating the eventual `Target` for modelling
df_combined['Target'] = df_combined['critical_success'] + ' ' +\
                        df_combined['financial_success'] + ' ' +\
                        df_combined['Economic Climate']

In [ ]:
# Creating a list of features to drop
cols_to_drop = [
    'Date',
    'CCI Rolling Mean',
    'CPI Rolling Mean',
    'Unemployment Rate (%) Rolling Mean',
    'Economic Climate',
    'Year',
    'Month',
    'id',
    'cast',
    'original_language',
    'director',
    'writers',
    'producers',
    'popularity', 
    'critical_success',
    'financial_success',
    'release_date',
    'released_day',
    'production_countries',
    'status',
    'spoken_languages'
]

# Dropping unneeded features
df_combined.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# Dropping `NaN` records
df_combined.dropna(inplace=True)

In [ ]:
# Confirming total records after concatenation
print(f'Total records: {df_combined.shape[0]}')

# Train Test Splitting

In [ ]:
# Defining columns to scale and encode
col_to_scale = [
    'CCI Value', 'CCI Rolling Percent Change', 'CPI Value',
    'CPI Rolling Percent Change', 'Unemployment Rate (%)', 
    'Unemployment Rate Rolling Percent Change','vote_average', 'vote_count',
    'revenue','runtime','budget', 'roi'
]

col_to_encode = [
    'CCI Rolling Percent Change Flag', 'CPI Rolling Percent Change Flag',
    'Unemployment Rate Rolling Percent Change Flag', 'title', 'original_title',
    'genres', 'production_companies'
]

# Setup X and y variables
X = df_combined.drop(columns='Target')
y = df_combined['Target']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

# Scaling and Econding

In [ ]:
# Creating an instance for `StandardScalar()`
scaler = StandardScaler()

# Fitting and transforming to `col_to_scale`
X_train_scaled = scaler.fit_transform(X_train[col_to_scale])
X_test_scaled = scaler.transform(X_test[col_to_scale])

# Converting results to DF for later concatenation
X_train_scaled = pd.DataFrame(X_train_scaled, columns=col_to_scale)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=col_to_scale)

In [ ]:
# Creating an instance for `OneHotEncoder()` for `X_train[col_to_encode]`
encoder_x = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fitting to `col_to_encode`
encoder_x.fit(X_train[col_to_encode])

# Creating an instance for `OneHotEncoder()` for `y_train`
encoder_y = LabelEncoder()

#Fitting
encoder_y.fit(y_train.values.reshape(-1,1))

# Transforming `X_train[col_to_encode]` and `X_test[col_to_encode]`
X_train_encoded = encoder_x.transform(X_train[col_to_encode])
X_test_encoded = encoder_x.transform(X_test[col_to_encode])

# Transforming `y_train` and `y_test`
y_train_encoded = encoder_y.transform(y_train.values.reshape(-1,1))
y_test_encoded = encoder_y.transform(y_test.values.reshape(-1,1))

# Converting results to DF for later concatenation
X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder_x.get_feature_names_out())
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder_x.get_feature_names_out())

In [ ]:
# Concatenating the `col_to_scale` with `col_to_encode` for `X_train` and `X_test`
X_train = pd.concat([X_train_scaled, X_train_encoded], axis=1)
X_test = pd.concat([X_test_scaled, X_test_encoded], axis=1)

In [ ]:
# Confirming total records after concatenation
print(f'Total X records: {X_train.shape[0] + X_test.shape[0]}')

# Modeling

Playtime!!

# Eric's Space

# Funda's Space

# Kalvin's Space

# Odele's Space

#### Peta-Gaye's LR modelling

In [ ]:
# Declare a logistic regression model.
logistic_regression_model = LogisticRegression()

In [ ]:
# Fit and save the logistic regression model using the training data
df_combined_lr_model = logistic_regression_model.fit(X_train, y_train_encoded)

In [ ]:
# Generate predictions from the logistic regression model using the test data
lr_predictions = logistic_regression_model.predict(X_test)

# Review the predictions
lr_predictions

In [ ]:
# Display the accuracy score for the test dataset.
accuracy_score(y_test_encoded, lr_predictions)

In [ ]:
# Display the precision score for the test dataset.
precision_score(y_test_encoded, lr_predictions, average='weighted')

#### End Peta-Gaye's LR modelling

# Peta's Space

# Vadim's Space

# Findings

# Additional